### Package Installation

To get started, we need to install the necessary packages. 

In [ ]:
!pip install ultralytics optuna torchvision scikit-learn pillow datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## Explanation of the Code

This code is part of a pipeline for training a YOLO (You Only Look Once) model to detect brain tumors using a dataset. Below is a detailed explanation of the key components and functions:

### Imports
- `os`, `shutil`: For file and directory operations.
- `torch`: PyTorch library for deep learning.
- `optuna`: For hyperparameter optimization.
- `sklearn.model_selection.KFold`: For K-Fold cross-validation.
- `ultralytics.YOLO`: YOLO model from the Ultralytics library.
- `torchvision.transforms`: For data augmentation.
- `PIL.Image`: For image processing.
- `datasets.load_dataset`: For loading datasets.
- `collections.Counter`: For counting occurrences of elements.

### Constants
- `BASE_DIR`: The current working directory.
- `OUTPUT_DIR`: Directory to store YOLO output.
- `CLASS_NAMES`: List of class names for brain tumor types.
- `PRETRAINED_WEIGHTS`: URL for downloading pretrained weights.

### Data Augmentation
- `augmentation_transforms`: A composition of several data augmentation techniques including random flips, rotations, color jitter, and resized cropping.

### Functions
1. **`save_data_yaml(output_dir, num_classes)`**:
   - Saves a YOLO-compatible `data.yaml` file with paths to training and testing images, number of classes, and class names.

2. **`process_dataset_split(dataset_split, split_name)`**:
   - Processes each dataset split (train or test) to create a class-based folder structure and saves images accordingly.

3. **`calculate_label_distribution(dataset_split, split_name)`**:
   - Calculates and displays the percentage distribution of labels in a dataset split using a `Counter`.

4. **`objective(trial)`**:
   - Defines an Optuna objective function for hyperparameter tuning. It tunes learning rate, batch size, and number of epochs, trains the YOLO model, and returns validation accuracy.

5. **`tune_hyperparameters()`**:
   - Performs hyperparameter optimization using Optuna, running multiple trials to find the best parameters for training the model.

This code sets up the necessary environment, processes the dataset, and optimizes the model's hyperparameters to achieve the best performance in detecting brain tumors.

In [ ]:
import os
import shutil
import torch
import optuna
from sklearn.model_selection import KFold
from ultralytics import YOLO
from torchvision import transforms
from PIL import Image
from datasets import load_dataset
from collections import Counter

# Constants
BASE_DIR = os.getcwd()
OUTPUT_DIR = os.path.join(BASE_DIR, 'brain_tumor_yolo')
os.makedirs(OUTPUT_DIR, exist_ok=True)
CLASS_NAMES = ["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"]

# Data augmentation transforms
augmentation_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor()
])


def save_data_yaml(output_dir, num_classes):
    """
    Save a YOLO-compatible data.yaml file and return its path.
    """
    data_yaml = {
        "train": os.path.join(output_dir, "train", "images"),
        "test": os.path.join(output_dir, "test", "images"),
        "nc": num_classes,
        "names": CLASS_NAMES
    }
    yaml_path = os.path.join(output_dir, "data.yaml")
    with open(yaml_path, 'w') as yaml_file:
        yaml_file.write(f"train: {data_yaml['train']}\n")
        yaml_file.write(f"test: {data_yaml['test']}\n")
        yaml_file.write(f"nc: {data_yaml['nc']}\n")
        yaml_file.write("names:\n")
        for name in data_yaml["names"]:
            yaml_file.write(f"  - {name}\n")
    return yaml_path


def process_dataset_split(dataset_split, split_name):
    """
    Process each dataset split (train or test) to create a class-based folder structure.
    """
    split_dir = os.path.join(OUTPUT_DIR, split_name)
    os.makedirs(split_dir, exist_ok=True)

    for class_name in CLASS_NAMES:
        class_dir = os.path.join(split_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)

    for idx, example in enumerate(dataset_split):
        image = example["image"]
        class_label = int(example["label"])
        class_name = CLASS_NAMES[class_label]
        image_name = f"{idx:05d}_{class_name}.png"
        image_path = os.path.join(split_dir, class_name, image_name)
        image.save(image_path)


def calculate_label_distribution(dataset_split, split_name):
    """
    Calculate and display the percentage distribution of labels in a dataset split.
    """
    label_counter = Counter()
    for idx, example in enumerate(dataset_split):
        class_label = int(example["label"])
        label_counter[class_label] += 1

    total_count = sum(label_counter.values())
    for class_label, count in label_counter.items():
        percentage = (count / total_count) * 100
        print(f"{CLASS_NAMES[class_label]}: {count} ({percentage:.2f}%)")

    return label_counter


def objective(trial):
    """
    Optuna objective function for hyperparameter tuning.
    """
    # Hyperparameters to tune
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    epochs = trial.suggest_int("epochs", 50, 150)

    # Train the model
    model = YOLO("yolov8n-cls.pt")
    model.train(
        data=os.path.join(OUTPUT_DIR),
        epochs=epochs,
        imgsz=224,
        batch=batch_size,
        lr0=learning_rate,
        device="cuda" if torch.cuda.is_available() else "cpu",
    )
    # Validation metrics
    metrics = model.val()
    return metrics["accuracy"]


def tune_hyperparameters():
    """
    Perform hyperparameter optimization using Optuna.
    """
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)
    print(f"Best parameters: {study.best_params}")
    return study.best_params


## Dataset Loading and Processing

This section of the code is responsible for loading a dataset from the Hugging Face Hub, calculating the label distribution, and processing the dataset into a class-based folder structure.

### Functions

1. **`load_dataset`**:
   - This function loads a dataset from the Hugging Face Hub or a local dataset. It supports various formats like JSON, CSV, Parquet, etc.
   - It takes multiple parameters to customize the loading process, such as `path`, `name`, `data_files`, `split`, `cache_dir`, etc.
   - It returns a `DatasetDict` or `Dataset` object containing the dataset splits.

2. **`calculate_label_distribution`**:
   - This function calculates and displays the percentage distribution of labels in a dataset split.
   - It iterates through the dataset split, counts the occurrences of each label, and prints the distribution.

3. **`process_dataset_split`**:
   - This function processes each dataset split (train or test) to create a class-based folder structure.
   - It creates directories for each class and saves the images in the corresponding class directory.

4. **`load_and_process_dataset`**:
   - This function orchestrates the loading and processing of the dataset.
   - It loads the dataset using `load_dataset`, calculates the label distribution for the training and testing splits using `calculate_label_distribution`, and processes the splits using `process_dataset_split`.

### Execution

- The `load_and_process_dataset` function is called to load the dataset, calculate label distributions, and process the dataset splits.
- After processing, a message is printed indicating that the dataset has been processed and saved to the specified output directory.

In [ ]:
# Load and process the dataset
def load_and_process_dataset():
    dataset = load_dataset("sartajbhuvaji/Brain-Tumor-Classification")
    train_split = "Training"
    test_split = "Testing"

    # Display label distributions
    print("Training Split Label Distribution:")
    calculate_label_distribution(dataset[train_split], train_split)
    print("\nTesting Split Label Distribution:")
    calculate_label_distribution(dataset[test_split], test_split)

    # Process the splits
    process_dataset_split(dataset[train_split], "train")
    process_dataset_split(dataset[test_split], "test")

load_and_process_dataset()
print(f"Dataset processed and saved to {OUTPUT_DIR}")


Training Split Label Distribution:
glioma_tumor: 826 (28.78%)
meningioma_tumor: 822 (28.64%)
no_tumor: 395 (13.76%)
pituitary_tumor: 827 (28.82%)

Testing Split Label Distribution:
glioma_tumor: 100 (25.38%)
meningioma_tumor: 115 (29.19%)
no_tumor: 105 (26.65%)
pituitary_tumor: 74 (18.78%)
Dataset processed and saved to /content/brain_tumor_yolo


# Generate the YAML file

In [ ]:
# Save the data.yaml file
yaml_path = save_data_yaml(OUTPUT_DIR, num_classes=4)
print(f"data.yaml saved to: {yaml_path}")


data.yaml saved to: /content/brain_tumor_yolo/data.yaml


## Model Training

This section of the code is responsible for training the YOLO model using the optimized hyperparameters. The key components of this cell include:

- **Model Initialization**: The YOLO classification model (`yolov8n-cls.pt`) is initialized.
- **Training Parameters**: The model is trained with the following parameters:
    - `data`: Path to the dataset directory.
    - `epochs`: Number of training epochs (set to 100).
    - `imgsz`: Image size (set to 224).
    - `batch`: Batch size (set to 32).
    - `lr0`: Initial learning rate (set to 0.005).
    - `device`: Device to use for training (CUDA if available, otherwise CPU).
    - `optimizer`: Optimizer used for training (AdamW for better regularization).
    - `weight_decay`: Regularization parameter (set to 1e-4).

After training, the model is saved to the specified output directory.

In [ ]:
# Train the YOLO model with optimized parameters
model = YOLO("yolov8n-cls.pt")  # Use YOLO classification model
model.train(
    data=OUTPUT_DIR,
    epochs=100,
    imgsz=224,
    batch=32,
    lr0=0.005,
    device="cuda" if torch.cuda.is_available() else "cpu",
    optimizer="AdamW",  # Using AdamW optimizer for better regularization
    weight_decay=1e-4  # Regularization parameter
)


Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/brain_tumor_yolo, epochs=100, time=None, patience=100, batch=32, imgsz=224, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sh

train: Scanning /content/brain_tumor_yolo/train... 2870 images, 0 corrupt: 100%|██████████| 2870/2870 [00:00<?, ?it/s]
val: Scanning /content/brain_tumor_yolo/test... 394 images, 0 corrupt: 100%|██████████| 394/394 [00:00<?, ?it/s]


optimizer: AdamW(lr=0.005, momentum=0.937) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0001), 27 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 224 train, 224 val
Using 2 dataloader workers
Logging results to runs/classify/train5
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size


      1/100     0.482G     0.7201         22        224: 100%|██████████| 90/90 [00:35<00:00,  2.51it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 7/7 [00:02<00:00,  3.24it/s]

                   all      0.629          1


...

                   all      0.812          1



100 epochs completed in 1.104 hours.
Optimizer stripped from runs/classify/train5/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/train5/weights/best.pt, 3.0MB

Validating runs/classify/train5/weights/best.pt...
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-cls summary (fused): 73 layers, 1,440,004 parameters, 0 gradients, 3.3 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /content/brain_tumor_yolo/train... found 2870 images in 4 classes ✅ 
val: None...
test: /content/brain_tumor_yolo/test... found 394 images in 4 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 7/7 [00:01<00:00,  3.86it/s]


                   all      0.863          1
Speed: 0.1ms preprocess, 0.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train5


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x77fd6e1eaf20>
curves: []
curves_results: []
fitness: 0.9314720928668976
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8629441857337952, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9314720928668976}
save_dir: PosixPath('runs/classify/train5')
speed: {'preprocess': 0.06954016419231589, 'inference': 0.21480727316764406, 'loss': 0.0002493107984513801, 'postprocess': 0.00023236734612944163}
task: 'classify'
top1: 0.8629441857337952
top5: 1.0

## Model Evaluation

In [ ]:
# Evaluate the model on the test split
model.val(split="test")

Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-cls summary (fused): 73 layers, 1,440,004 parameters, 0 gradients, 3.3 GFLOPs
train: /content/brain_tumor_yolo/train... found 2870 images in 4 classes ✅ 
val: None...
test: /content/brain_tumor_yolo/test... found 394 images in 4 classes ✅ 


test: Scanning /content/brain_tumor_yolo/test... 394 images, 0 corrupt: 100%|██████████| 394/394 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 13/13 [00:05<00:00,  2.39it/s]


                   all      0.863          1
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train52


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x77fd604b5ff0>
curves: []
curves_results: []
fitness: 0.9314720928668976
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8629441857337952, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9314720928668976}
save_dir: PosixPath('runs/classify/train52')
speed: {'preprocess': 0.05773420866370806, 'inference': 0.9387247453486254, 'loss': 0.00045565784280070194, 'postprocess': 0.0005228265287912436}
task: 'classify'
top1: 0.8629441857337952
top5: 1.0